# get runs

In [1]:
import numpy as np

In [2]:

WANDB_ENTITY = "mines-paristech-cmm"
WANDB_PROJECT = "unetdd-mvtec-00"
WANDB_ENTITY_PROJECT = f"{WANDB_ENTITY}/{WANDB_PROJECT}"

WANDB_SWEEP_ID = "73omzk4h"
WANDB_SWEEP_PATH = f"{WANDB_ENTITY}/{WANDB_PROJECT}/{WANDB_SWEEP_ID}"

In [3]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
# runs = api.runs(WANDB_ENTITY_PROJECT)

sweep = api.sweep(WANDB_SWEEP_PATH)
runs = sweep.runs

In [4]:
summary_list, config_list, name_list = [], [], []
tags_list = []
ids_list = []
state_list = []

for run in runs: 
    summary_list.append(run.summary._json_dict)
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})
    name_list.append(run.name)
    tags_list.append(run.tags)
    ids_list.append(run.id)
    state_list.append(run.state)
    
runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list,
    'tags': tags_list,
    'id': ids_list,
    'state': state_list,
})

In [5]:
runs_df.shape

(116, 6)

In [6]:
runs_df.columns

Index(['summary', 'config', 'name', 'tags', 'id', 'state'], dtype='object')

# filter runs

In [7]:
runs_df_filtered = runs_df

get_only_finished = runs_df['state'] == 'finished'
runs_df_filtered = runs_df_filtered[get_only_finished]

select_epoch_is_last = runs_df["summary"].apply(lambda s: s.get("epoch", None)) == 600
# select_epoch_is_last = runs_df["summary"].apply(lambda s: s.get("epoch", None)) == 500
runs_df_filtered = runs_df_filtered[select_epoch_is_last]

runs_df_filtered.shape

(116, 6)

In [8]:
runs_df_filtered["summary"].iloc[0].keys()


dict_keys(['_wandb', 'normal_class_label', 'train/pixel-count-normal', 'train/loss-anomalous-mean', 'test/score-mean', 'trainer/global_step', 'train/pixel-count-anomalous', 'validate/images-heatmaps-table', 'test/precision-recall-curve_table', 'train/loss', 'test/score-anomalous-mean', 'validate/score-anomalous-mean', '_step', 'test/roc-auc', 'validate/score-mean', 'test/loss-normals-mean', 'test/pixel-count-normal', '_runtime', 'test/loss', 'test/avg-precision', 'train/score-normals-mean', 'test/pixel-count-anomalous', 'validate/loss-normals-mean', 'train/score-mean', 'validate/roc-auc', 'train/loss-normals-mean', 'validate/score-normals-mean', 'validate/loss-anomalous-mean', 'normal_class_fullqualified', 'train/score-anomalous-mean', 'epoch', 'mvtec_class_type', 'test/roc-curve_table', 'validate/avg-precision', 'test/score-normals-mean', 'test/loss-anomalous-mean', 'validate/pixel-count-normal', '_timestamp', 'validate/loss', 'batch_size_effective', 'test/images-heatmaps-table', 'val

In [9]:
runs_df_filtered["config"].iloc[0].keys()

dict_keys(['it', 'lr', 'pid', 'loss', 'root', 'seed', 'test', 'model', 'seeds', 'epochs', 'logdir', 'classes', 'datadir', 'dataset', 'in_shape', 'nworkers', 'loss_name', 'net_shape', 'optimizer', 'raw_shape', 'scheduler', 'seeds_str', 'batch_size', 'model_name', 'pin_memory', 'wandb_watch', 'normal_class', 'slurm_job_id', 'wandb_log_pr', 'weight_decay', 'learning_rate', 'preprocessing', 'wandb_log_roc', 'wandb_profile', 'optimizer_name', 'scheduler_name', 'script_version', 'slurm_job_gpus', 'slurm_job_name', 'slurm_job_user', 'slurm_task_pid', 'supervise_mode', 'confighash_full', 'preview_nimages', 'slurmd_nodename', 'confighash_slurm', 'script_start_time', 'slurm_submit_host', 'lightning_ndevices', 'lightning_profiler', 'lightning_strategy', 'loss_stage_weights', 'real_anomaly_limit', 'slurm_array_job_id', 'slurm_cluster_name', 'lightning_precision', 'slurm_array_task_id', 'slurm_job_partition', 'scheduler_parameters', 'wandb_watch_log_freq', 'lightning_accelerator', 'loss_scoremap_st

# find missing ones

In [10]:
runs_df_filtered["class"] = runs_df_filtered["config"].apply(lambda c: c.get("normal_class", None))
runs_df_filtered["supervise_mode"] = runs_df_filtered["config"].apply(lambda c: c.get("supervise_mode", None))
runs_df_filtered["seeds"] = runs_df_filtered["config"].apply(lambda c: c.get("seeds", None))
runs_df_filtered[["class", "supervise_mode"]].isnull().any(axis=1).any()
runs_df_filtered

,summary,config,name,tags,id,state,class,supervise_mode,seeds
0,"{'_wandb': {'runtime': 14141}, 'normal_class_l...","{'it': 0, 'lr': 0.002, 'pid': 922261, 'loss': ...",fiery-sweep-138,"[dev01, sweep-02, u2net]",tu807wlm,finished,14,real-anomaly,0x22cd096d5f8a9c21
1,{'train/loss-normals-mean': 0.1153688356280326...,"{'it': 0, 'lr': 0.002, 'pid': 904568, 'loss': ...",noble-sweep-137,"[dev01, sweep-02, u2net]",gmuzesih,finished,14,synthetic-anomaly-confetti,0x22cd096d5f8a9c21
2,"{'test/score-mean': 8.427359580993652, 'valida...","{'it': 0, 'lr': 0.002, 'pid': 897731, 'loss': ...",firm-sweep-136,"[dev01, sweep-02, u2net]",w63y4aua,finished,14,real-anomaly,0x2395859d21efbbd6
3,"{'validate/roc-auc': 0.519996930554105, 'train...","{'it': 0, 'lr': 0.002, 'pid': 886951, 'loss': ...",warm-sweep-135,"[dev01, sweep-02, u2net]",fdn2ikor,finished,14,synthetic-anomaly-confetti,0x2395859d21efbbd6
4,"{'_runtime': 10767, 'test/loss': 20.3888854980...","{'it': 0, 'lr': 0.002, 'pid': 3298177, 'loss':...",northern-sweep-134,"[dev01, sweep-02, u2net]",l9n0x09e,finished,14,real-anomaly,0x40452d6f87068d81
...,...,...,...,...,...,...,...,...,...
111,{'test/score-normals-mean': 0.0955930277705192...,"{'it': 0, 'lr': 0.002, 'pid': 20941, 'loss': '...",peachy-sweep-5,"[dev01, sweep-02, u2net]",5mjhurzm,finished,0,synthetic-anomaly-confetti,0x2395859d21efbbd6
112,"{'validate/loss': 23.034969329833984, 'batch_s...","{'it': 0, 'lr': 0.002, 'pid': 19699, 'loss': '...",giddy-sweep-4,"[dev01, sweep-02, u2net]",uhbaskkd,finished,0,real-anomaly,0x40452d6f87068d81
113,{'train/loss-normals-mean': 0.1313494592905044...,"{'it': 0, 'lr': 0.002, 'pid': 1014, 'loss': 'u...",rare-sweep-3,"[dev01, sweep-02, u2net]",uycvoaqh,finished,0,synthetic-anomaly-confetti,0x40452d6f87068d81
114,"{'_timestamp': 1658249661, 'trainer/global_ste...","{'it': 0, 'lr': 0.002, 'pid': 7884, 'loss': 'u...",misunderstood-sweep-2,"[dev01, sweep-02, u2net]",aufwi3yu,finished,0,real-anomaly,0x526f289f74732498


In [13]:
all_seeds = {
    "0x526f289f74732498",
    "0x40452d6f87068d81",
    "0x2395859d21efbbd6",
    "0x22cd096d5f8a9c21",
    "0xeb33f3d140c7b5f4",   
}

missing_df = pd.DataFrame()
missing_df["seeds"] = runs_df_filtered.groupby(["class", "supervise_mode"])["seeds"].apply(lambda s: all_seeds - set(s))
missing_df["n"] = missing_df["seeds"].apply(lambda s: len(s))
print(f"total missing runs: {missing_df['n'].sum()}")
missing_df = missing_df[missing_df["n"] > 0].drop(columns=["n"])
missing_df.reset_index(inplace=True)
missing_df

total missing runs: 29


,class,supervise_mode,seeds
0,0,real-anomaly,{0x2395859d21efbbd6}
1,1,synthetic-anomaly-confetti,"{0xeb33f3d140c7b5f4, 0x22cd096d5f8a9c21, 0x404..."
2,2,synthetic-anomaly-confetti,{0x2395859d21efbbd6}
3,3,real-anomaly,"{0x22cd096d5f8a9c21, 0x40452d6f87068d81}"
4,3,synthetic-anomaly-confetti,"{0x526f289f74732498, 0xeb33f3d140c7b5f4}"
5,4,real-anomaly,{0x526f289f74732498}
6,4,synthetic-anomaly-confetti,"{0xeb33f3d140c7b5f4, 0x40452d6f87068d81}"
7,5,real-anomaly,{0x22cd096d5f8a9c21}
8,6,real-anomaly,{0x2395859d21efbbd6}
9,6,synthetic-anomaly-confetti,{0xeb33f3d140c7b5f4}


In [12]:
missing_args = [
    f"--supervise_mode {supervise_mode} --classes {class_} --seeds {seed}"
    for class_, supervise_mode, seeds in missing_df.itertuples(index=False)
    for seed in seeds
]
print("\n".join(missing_args))

--supervise_mode real-anomaly --classes 0 --seeds 0x2395859d21efbbd6
--supervise_mode synthetic-anomaly-confetti --classes 1 --seeds 0xeb33f3d140c7b5f4
--supervise_mode synthetic-anomaly-confetti --classes 1 --seeds 0x22cd096d5f8a9c21
--supervise_mode synthetic-anomaly-confetti --classes 1 --seeds 0x40452d6f87068d81
--supervise_mode synthetic-anomaly-confetti --classes 2 --seeds 0x2395859d21efbbd6
--supervise_mode real-anomaly --classes 3 --seeds 0x22cd096d5f8a9c21
--supervise_mode real-anomaly --classes 3 --seeds 0x40452d6f87068d81
--supervise_mode synthetic-anomaly-confetti --classes 3 --seeds 0x526f289f74732498
--supervise_mode synthetic-anomaly-confetti --classes 3 --seeds 0xeb33f3d140c7b5f4
--supervise_mode real-anomaly --classes 4 --seeds 0x526f289f74732498
--supervise_mode synthetic-anomaly-confetti --classes 4 --seeds 0xeb33f3d140c7b5f4
--supervise_mode synthetic-anomaly-confetti --classes 4 --seeds 0x40452d6f87068d81
--supervise_mode real-anomaly --classes 5 --seeds 0x22cd096d